In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *

from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.plotting.plotting_base import plot_or_results


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe="5m", candles_to_dl=550)

In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
# backtest_settings = BacktestSettings(
#     array_size=200,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([5]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=True,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type=TakeProfitFeeType.Limit,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)
strategy = Strategy(
    rsi_is_below=np.array([70]),
    rsi_length=np.array([14]),
    long_short="long",
)

In [9]:
run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 1
Total order settings to test: 1
Total combinations of settings to test: 1
Total candles: 550
Total candles to test: 550


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,18.0,4,14,-29.215,22.222,-0.408,643.375,-292.147,707.853


In [12]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=0,
    ind_set_index=0,
    plot_results=True,
)

In [13]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,0,0,50,1699988700000,2023-11-14 19:05:00,EntryFilled,1000.000,961.175,-38.774,38.825,...,1800.888,35180.6,NaN,0.051,1800.888,NaN,1.500,34657.4,8.5,38154.0
1,0,0,51,1699989000000,2023-11-14 19:10:00,EntryFilled,1000.000,888.842,-149.845,111.158,...,1288.481,35419.0,NaN,0.087,3089.369,NaN,1.700,34657.4,9.8,38716.8
2,0,0,52,1699989300000,2023-11-14 19:15:00,EntryFilled,1000.000,777.995,-371.722,222.005,...,1440.062,35393.5,NaN,0.128,4529.431,NaN,1.800,34657.4,10.1,38833.8
3,0,0,53,1699989600000,2023-11-14 19:20:00,EntryFilled,1000.000,628.964,-742.589,371.036,...,1438.689,35365.8,NaN,0.169,5968.120,NaN,1.800,34657.4,10.2,38892.8
4,0,0,206,1700035500000,2023-11-15 08:05:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.018,35480.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0,0,546,1700137500000,2023-11-16 12:25:00,StopLossFilled,728.839,728.839,NaN,NaN,...,NaN,NaN,37109.4,NaN,NaN,-21.988,NaN,NaN,NaN,NaN
99,0,0,547,1700137800000,2023-11-16 12:30:00,EntryFilled,728.839,637.823,-90.726,91.016,...,10764.661,37095.0,NaN,0.290,10764.661,NaN,NaN,37089.4,1.1,37501.5
100,0,0,547,1700137800000,2023-11-16 12:30:00,StopLossFilled,707.853,707.853,NaN,NaN,...,NaN,NaN,37089.4,NaN,NaN,-20.986,NaN,NaN,NaN,NaN
101,0,0,548,1700138100000,2023-11-16 12:35:00,EntryFilled,707.853,670.087,-37.676,37.766,...,3344.934,37236.3,NaN,0.090,3344.934,NaN,0.400,37069.4,3.3,38450.2
